# Un'analisi IRT dei quesiti del Bebras italiano

L'*Item Response Theory* (IRT) è una tecnica statistica che si propone di stimare l'*abilità* dei soggetti cui viene sottoposto un test composto da una serie di domande (dette *item*). L'abilità è un tratto *latente*, cioè non osservabile direttamente: l'aver risposto correttamente a molte domande può dipendere sia da elevata abilità, che dalla "facilità" delle domande. La stima si basa quindi su un *modello matematico* (detto *Item Response Function*) della relazione fra abilità e probabilità di rispondere correttamente alla domanda.

Il modello usato in questa analisi è una curva *logistica*, la cui forma è regolata da tre parametri $a, b, c$.
$$ p_i(\theta_j) = c_i + \frac{(1 - c_i)}{1+ e^{-b_i\cdot(\theta_j - a_i)}} $$
La probabilità $p$ di successo in un *item* $i$ è funzione dell'abilità $\theta$ del solutore $j$. I tre parametri modellano le caratteristiche dell'*item*:
- $a$, detto *difficoltà* dell'item: determina il livello di abilità necessaria per avere il 50% di probabilità di successo;
- $b$, detto *differenziazione*: determina quanto influiscono le variazioni di abilità sulla probabilità di successo;
- $c$: fissa una probabilità minima di successo per qualsiasi abilità, dovuta per esempio alla possibilità di scegliere casualmente la risposta giusta ($c$ sarà 0 per un *item* a risposta completamente aperta)

In [ ]:
logistic <- function(theta, a=0, b=1, c=0) { 
    c + (1-c)/(1 + exp(-b*(theta-a)))
}

In [ ]:
library("ggplot2")
data <- data.frame(x=seq(-5,5,.1))
data <- transform(data, 
                  y_mean = logistic(x), 
                  y_hard = logistic(x,a=2), 
                  y_easy=logistic(x,a=-2),
                  y_hdiscrimination=logistic(x,b=2),
                  y_ldiscrimination=logistic(x,b=.5)
                 );

options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_line(aes(y=y_hard), color="red")
p <- p + annotate("text", x=2.1, y=.55, label="a == 2 ~~(plain(difficile))", parse=TRUE, color="red")
p <- p + geom_line(aes(y=y_easy), color="dark green")
p <- p + annotate("text", x=-1.9, y=.55, label="a == -2 ~~(plain(facile))", parse=TRUE, color="dark green")
p <- p + geom_line(aes(y=y_mean), color="blue")
p <- p + annotate("text", x=1.5, y=.8, label="a == 0 ~~(plain(medio))", parse=TRUE, color="blue")
p <- p + geom_line(aes(y=y_hdiscrimination), color="dark cyan")
p <- p + annotate("text", x=1, y=.9, label="a == 0 ~~ b == 2", parse=TRUE, color="dark cyan")
p <- p + geom_line(aes(y=y_ldiscrimination), color="violet")
p <- p + annotate("text", x=1.8, y=.7, label="a == 0 ~~ b == .5", parse=TRUE, color="violet")
p <- p + labs(title=expression(paste("Curve logistiche per diversi valori di ", a, " e ", b, " (",c==0,")")), 
              x="Abilità", y="Probabilità di successo")

p
rm(data, p)

A ogni *item* o quiz, quindi, viene associata una specifica curva logistica (la sua *item response function*) che può essere dedotta osservando un campione di risolutori cui è stato sottoposto il test. A questo scopo i parametri relativi ai quiz del Bebras italiano 2015 sono stati stimati con un approccio bayesiano, come descritto in [C. Bellettini et al. *"How challenging are Bebras tasks? an IRT analysis based on the performance of Italian students", Proceedings of the 20th annual conference on innovation and technology in computer science education ITiCSE'15* (Vilnius, Lithuania, 2015)](http://dx.doi.org/10.1145/2729094.2742603). I dettagli statistici riguardo la bontà di adattamento del modello sono disponibili [qui](https://mmonga.shinyapps.io/bebrasIT2015).

## I quesiti 2015

In [ ]:
library("rstan"); rstan_options(auto_write = TRUE); options(mc.cores = parallel::detectCores())
load("fit.RData") # carica l'oggetto fit
dfit <- as.data.frame(fit)
obs <- read_rdump("bebras-all.data.R")
source("qnames.R")

cats <- c("kilo", "mega", "giga", "tera", "peta")
categories <- factor(cats, levels=cats, ordered = TRUE)
rm(cats)

`dfit` è il data frame che raccoglie l'esito della simulazione, con 3394 variabili (3168 *ability*, 75 *discrimination*, 75 *difficulty*, 75 *guessing* e la logprobability) e 2000 campioni. `obs` è la lista dei dati effettivamente osservati, cioè 47520 risposte di 3168 squadre a 75 quiz, con 5 categorie. `qnames` contiene i nomi completi dei quiz, come `'2015_Kilo_A1_HU02_Apparecchiare'`.

In [ ]:
obs$rcats <- plyr::aaply(obs$rquiz, 1, function (e){ as.character(categories[ ((e - 1) %/% 15) + 1 ]) })

In [ ]:
library("plyr")
library("dplyr")
teams <- data.frame(id = obs$rtaker, category = obs$rcats) %>% distinct()
get_par_df <- function(df, par, table) {
    df %>% 
    dplyr::select(contains(par)) %>% 
    tidyr::gather(key = "parameter", value, everything()) %>%
    dplyr::mutate("id" = as.numeric(regmatches(parameter, regexpr("[0-9]+",parameter)))) %>%
    dplyr::left_join(table, by = "id") %>%
    transform(category = factor(category, levels=levels(categories), ordered = TRUE))
}

In [ ]:
team_ability <- get_par_df(dfit, "ability", teams)

Il modello è tarato in modo che la media delle abilità di tutte le squadre partecipanti sia 0.

In [ ]:
print(c(summary(team_ability$value), paste("StdDev: ", sd(team_ability$value))), quote = FALSE)

In [ ]:
team_ability.notes <- ddply(team_ability, .(category), summarise, note=sprintf("paste(mu == %s, ' ', sigma == %s)", 
                                                                     format(mean(value), digit = 2), 
                                                                     format(sd(value), digit= 2)))

In [ ]:
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(team_ability, aes(x=value)) 
p <- p + geom_histogram(binwidth=.2) + coord_cartesian(ylim = c(0,.5))
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
p <- p + geom_text(data=team_ability.notes, aes(0, 0.4, label=note), size=3, parse = TRUE)
p <- p + labs(title="Distribuzione dell'abilità dei partecipanti", 
              x="Abilità", y="Densità")
p

In [ ]:
qorder <- plyr::laply(strsplit(qnames, "_"), function(e){e[3]})
quizzes <- data.frame(id = 1:length(qnames), quiz = qnames, 
                      category = factor(plyr::laply(strsplit(qnames, "_"), function(e){tolower(e[2])}), 
                                        levels=levels(categories), ordered = TRUE),
                      qname = plyr::laply(strsplit(qnames, "_"), function(e){e[5]}),
                      bebras_id = plyr::laply(strsplit(qnames, "_"), function(e){e[4]}),
                      ord = factor(qorder, levels=rev(sort(unique(qorder))), ordered = TRUE),
                      country = plyr::laply(strsplit(qnames, "_"), function(e){substr(e[4],1,2)}),
                      block = plyr::laply(strsplit(qnames, "_"), function(e){substr(e[3],1,1)}))
                      

In [ ]:
quiz_difficulty <- get_par_df(dfit, par = "difficulty", table = quizzes)                                    

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(quiz_difficulty, aes(ord, value, fill=category)) 
p <- p + geom_boxplot(outlier.shape = 1, outlier.size = .5)
p <- p + coord_flip() 
p <- p + labs(title="Distribuzione della difficoltà dei quiz", 
              x="Quiz", y="Difficoltà")
p
rm(p)

In [ ]:
quiz_discrimination <- get_par_df(dfit, par = "discrimination", table = quizzes)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(quiz_discrimination, aes(ord, value, fill=category)) 
p <- p + geom_boxplot(outlier.shape = 1, outlier.size = .5)
p <- p + coord_flip() 
p <- p + labs(title="Distribuzione della differenziazione dei quiz", 
              x="Quiz", y="Differenziazione")
p
rm(p)

In [ ]:
quiz_guessing <- get_par_df(dfit, par = "guessing", table = quizzes)

##  I quesiti

###  2015_Kilo_A1_HU02_Apparecchiare

![Apparecchiare](screenshot/kilo01-collage.png)

In [ ]:
q <- 'HU02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
p
summary(d$value)

#### Differenziazione

In [ ]:
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
p
summary(a$value)

#### Guessing

In [ ]:
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
p
summary(g$value)

In [ ]:
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
for (i in 1:length(d$value)) {
  p <- p + geom_line(aes(y=logistic(x, a=d$value[i], b=a$value[i], c=g$value[i])))
}
p